In [ ]:
from tensorflow.keras import layers
import os
import numpy as np
import keras
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
class ChestXRayGenerator(Sequence):
    def __init__(self, directory, batch_size=4, img_size=512, shuffle=True):
        self.img_size = img_size
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.normal_dir = os.path.join(directory, 'NORMAL')
        self.pneumonia_dir = os.path.join(directory, 'PNEUMONIA')
        self.image_paths = []
        self.labels = []
        for fname in os.listdir(self.normal_dir):
            self.image_paths.append(os.path.join(self.normal_dir, fname))
            self.labels.append(0)  # NORMAL

        for fname in os.listdir(self.pneumonia_dir):
            self.image_paths.append(os.path.join(self.pneumonia_dir, fname))
            self.labels.append(1)  # PNEUMONIA

        self.image_paths = np.array(self.image_paths)
        self.labels = np.array(self.labels)
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_paths = self.image_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]

        batch_images = [
            img_to_array(load_img(p, target_size=(self.img_size, self.img_size), color_mode='grayscale')) / 255.0
            for p in batch_paths
        ]
        batch_images = np.expand_dims(batch_images, -1)
        return np.array(batch_images), np.array(batch_labels)

    def on_epoch_end(self):
        if self.shuffle:
            idxs = np.arange(len(self.image_paths))
            np.random.shuffle(idxs)
            self.image_paths = self.image_paths[idxs]
            self.labels = self.labels[idxs]



In [ ]:
img_size = 512
def get_model():
    inputs = layers.Input(shape=(img_size, img_size, 1))

    x = layers.Conv2D(32, (3, 3), strides=1, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D((2,2), strides=2, padding='same')(x)

    x = layers.Conv2D(64, (3,3), strides=1, padding='same', activation='relu')(x)
    x = layers.Dropout(0.1)(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D((2,2), strides=2, padding='same')(x)

    x = layers.Conv2D(64, (3,3), strides=1, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D((2,2), strides=2, padding='same')(x)

    x = layers.Conv2D(128, (3,3), strides=1, padding='same', activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D((2,2), strides=2, padding='same')(x)

    x = layers.Conv2D(256, (3,3), strides=1, padding='same', activation='relu', name="last_conv_layer")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D((2,2), strides=2, padding='same')(x)

    x = layers.Flatten()(x)
    x = layers.Dense(units=128, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    output = layers.Dense(units=1, activation='sigmoid')(x)

    model = keras.Model(inputs=inputs, outputs=output)
    return model
model = get_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
train_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
test_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'

train_gen = ChestXRayGenerator(train_dir, batch_size=2, img_size=512)
test_gen = ChestXRayGenerator(test_dir, batch_size=2, img_size=512, shuffle=False)


early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(train_gen, callbacks=[early_stopping], validation_data=test_gen, epochs=50)


In [ ]:
# Final values
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

print(f"Train Accuracy: {final_train_acc:.4f} | Validation Accuracy: {final_val_acc:.4f}")
print(f"Train Loss: {final_train_loss:.4f} | Validation Loss: {final_val_loss:.4f}")

In [ ]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b-', label='Train Accuracy')
    plt.plot(epochs, val_acc, 'r-', label='Val Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b-', label='Train Loss')
    plt.plot(epochs, val_loss, 'r-', label='Val Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_history(history)
